In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import math
import screeninfo
import freenect


In [2]:
%matplotlib
def plot(string='',*images):
    ncols=2
    nrows=int(math.ceil(len(images)/2.0))
    fig, ax = plt.subplots(nrows, ncols,gridspec_kw = {'wspace':0, 'hspace':0},facecolor='.5')
    plt.axis('off')
    for i in range(0,len(images)):
        plt.subplot(nrows,ncols,i+1),plt.imshow(images[i],cmap='gray')
        plt.text(0,0,string+'\n')
        plt.axis('off')
    plt.show()

Using matplotlib backend: TkAgg


In [3]:
def get_frame():
    array,_ = freenect.sync_get_video()
    array = cv2.cvtColor(array,cv2.COLOR_RGB2GRAY)
    return array

In [4]:
def find_dest(in_points,h):
    points=[]
    for point in in_points:
        point=list(point)
        point.append(1)
        homo_points=np.matmul(h,point)
        points.append([int(homo_points[0]/homo_points[2]),int(homo_points[1]/homo_points[2])])
    return np.array(points)

In [5]:
def make_rect_points(cv_rw,cv_cl):
    h=100
    w=150
    return [(cv_cl,cv_rw),(cv_cl+w,cv_rw),(cv_cl,cv_rw+h),(cv_cl+w,cv_rw+h)]

In [6]:
def find_white_sheets():
    kernel = np.ones((5,5),np.uint8)
    
    frame=cv2.imread("./test.jpg",1)
    plot('frame',frame)
    
    blur= cv2.medianBlur(frame, 11)
    edged=cv2.Canny(blur, 25, 200)
    dilation = cv2.dilate(edged,kernel,iterations = 1)
    plot('edged',dilation)
    im, contours, hierarchy = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    hierarchy=hierarchy[0]
    print hierarchy
    for i,each in enumerate(contours):
        if hierarchy[i][3]<0:
            image_prop_list = is_parent_return_image(i, hierarchy,contours,frame)
            break
    return image_prop_list    
        

In [7]:
def count_children(index, h_, contour):
    if h_[index][2] < 0:
        return 0
    else:
        count = 1
        # Also count all of the child's siblings and their children
        count += count_siblings(h_[index][2], h_, contour, True)
        return count

In [8]:
def count_siblings(index, h_, contour, inc_children=False):
    # Include the children if necessary
    if inc_children:
        count = count_children(index, h_, contour)
    else:
        count = 0

    # Look ahead
    p_ = h_[index][0]
    while p_ > 0:
        count += 1
        if inc_children:
            count += count_children(p_, h_, contour)
        p_ = h_[p_][0]

    # Look behind
    n = h_[index][1]
    while n > 0:
        count += 1
        if inc_children:
            count += count_children(n, h_, contour)
        n = h_[n][1]
    return count

In [9]:
def is_parent_return_image(index, h_,contour,img):
    img_childno_array=[]
    
    parent = h_[index][3]
    if parent < 0:
        p_ = index
        while p_ >= 0:

            rect = cv2.minAreaRect(contour[p_])
            W = rect[1][0]
            H = rect[1][1]
            if W*H >300:
                im_box = crop_im_box(rect,img)
                children_no = count_children(p_, h_, contour)
                img_childno_array.append([im_box,children_no])
            
            p_ = h_[p_][0]

        # Look behind
        n = h_[index][1]
        while n >= 0:
            rect = cv2.minAreaRect(contour[n])
            W = rect[1][0]
            H = rect[1][1]
            if W*H >300:
                im_box = crop_im_box(rect,img)
                children_no = count_children(p_, h_, contour)
                img_childno_array.append([im_box,children_no])
            
            n = h_[n][1]       
        return img_childno_array
    else :
        return False

In [10]:
 def crop_im_box(rect,img):
    mult = 1.2
    W = rect[1][0]
    H = rect[1][1]
    
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    Xs = [i[0] for i in box]
    Ys = [i[1] for i in box]
    x1 = min(Xs)
    x2 = max(Xs)
    y1 = min(Ys)
    y2 = max(Ys)
    temp=img.copy()
    cv2.drawContours(temp, [box], 0, (0,255,0), 2)
    rotated = False
    angle = rect[2]

    if W<H:
        angle+=90
        rotated = True

    center = (int((x1+x2)/2), int((y1+y2)/2))
    size = (int(mult*(x2-x1)),int(mult*(y2-y1)))

    M = cv2.getRotationMatrix2D((size[0]/2, size[1]/2), angle, 1.0)

    cropped = cv2.getRectSubPix(img, size, center)    
    cropped = cv2.warpAffine(cropped, M, size)

    croppedW = W if not rotated else H 
    croppedH = H if not rotated else W

    croppedRotated = cv2.getRectSubPix(cropped, (int(croppedW*mult), int(croppedH*mult)), (size[0]/2, size[1]/2))


    return [croppedRotated,box]

In [11]:
def show_anime(img):
    screen_id = 1
    # get the size of the screen
    screen = screeninfo.get_monitors()[screen_id]
    colms, rows = screen.width, screen.height
   
    window_name = 'anime'
    cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
    cv2.moveWindow(window_name, screen.x - 1, screen.y - 1)
    cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN,
                          cv2.WINDOW_FULLSCREEN)
    cv2.imshow(window_name, img)
    cv2.waitKey(0)

In [12]:
def takecenteroid(elem):
    M = cv2.moments(elem[0][1])
    cX = int(M["m10"] / M["m00"])
    return int(cX)


In [13]:
def mouse_handler(event, x, y, flags, data) :
    
    if event == cv2.EVENT_LBUTTONDOWN :
        cv2.circle(data['im'], (x,y),3, (0,0,255), 5, 16);
        cv2.imshow("Image", data['im']);
        if len(data['points']) < 4 :
            data['points'].append([x,y])

In [14]:
def get_four_points(im):
    
    # Set up data to send to mouse handler
    data = {}
    data['im'] = im.copy()
    data['points'] = []
    
    #Set the callback function for any mouse event
    cv2.imshow("Image",im)
    cv2.setMouseCallback("Image", mouse_handler, data)
    cv2.waitKey(0)
    
    # Convert array to np.array
    points = np.vstack(data['points']).astype(float)
    
    return points

In [15]:
def find_homo(src,dst):
    h, status = cv2.findHomography(src,dst)
    return h

In [20]:
image_and_box = find_white_sheets()
sortedImage = sorted(image_and_box, key=takecenteroid)
screen_id = 1
screen = screeninfo.get_monitors()[screen_id]
colms, rows = screen.width, screen.height

img = np.zeros((rows,colms,3),np.uint8)
temp=img.copy()
pts_dst = get_four_points(img)

read_h = np.load('homography.npy').item()
read_h['homo']
cv2.destroyAllWindows()
for i in sortedImage:
    box = find_dest(i[0][1],read_h['homo'])
    cv2.drawContours(temp, [box], 0, (255,255,255), 7)
    size=i[0][0].shape
    pts_src = np.array(
                       [
                        [0,0],
                        [size[1] - 1, 0],
                        [0, size[0] - 1 ],
                        [size[1] - 1, size[0] -1]
                        ],dtype=float)
    h = find_homo(pts_src,pts_dst)
    im_temp = cv2.warpPerspective(i[0][0], h, (colms,rows))
    new=im_temp+temp
    show_anime(new)
cv2.destroyAllWindows()


[[ 2 -1  1 -1]
 [-1 -1 -1  0]
 [ 4  0  3 -1]
 [-1 -1 -1  2]
 [ 6  2  5 -1]
 [-1 -1 -1  4]
 [ 8  4  7 -1]
 [-1 -1 -1  6]
 [10  6  9 -1]
 [-1 -1 -1  8]
 [12  8 11 -1]
 [-1 -1 -1 10]
 [14 10 13 -1]
 [-1 -1 -1 12]
 [16 12 15 -1]
 [-1 -1 -1 14]
 [-1 14 17 -1]
 [-1 -1 -1 16]]
